In [1]:
import torch
from pointnet2_ops import pointnet2_utils
import os

In [11]:
def square_distance(src, dst):
    """
    Calculate Euclid distance between each two points.
    src^T * dst = xn * xm + yn * ym + zn * zm；
    sum(src^2, dim=-1) = xn*xn + yn*yn + zn*zn;
    sum(dst^2, dim=-1) = xm*xm + ym*ym + zm*zm;
    dist = (xn - xm)^2 + (yn - ym)^2 + (zn - zm)^2
         = sum(src**2,dim=-1)+sum(dst**2,dim=-1)-2*src^T*dst
    Input:
        src: source points, [B, N, C]
        dst: target points, [B, M, C]
    Output:
        dist: per-point square distance, [B, N, M]
    """
    B, N, _ = src.shape
    _, M, _ = dst.shape
    dist = -2 * torch.matmul(src, dst.permute(0, 2, 1))
    dist += torch.sum(src ** 2, -1).view(B, N, 1)
    dist += torch.sum(dst ** 2, -1).view(B, 1, M)
    return dist

def farthest_point_sample(xyz, npoint):
    """
    Input:
        xyz: pointcloud data, [B, N, 3]
        npoint: number of samples
    Return:
        centroids: sampled pointcloud index, [B, npoint]
    """
    device = xyz.device
    B, N, C = xyz.shape
    centroids = torch.zeros(B, npoint, dtype=torch.long).to(device)
    distance = torch.ones(B, N).to(device) * 1e10
    farthest = torch.randint(0, N, (B,), dtype=torch.long).to(device)
    batch_indices = torch.arange(B, dtype=torch.long).to(device)
    for i in range(npoint):
        centroids[:, i] = farthest
        centroid = xyz[batch_indices, farthest, :].view(B, 1, 3)
        dist = torch.sum((xyz - centroid) ** 2, -1)
        mask = dist < distance
        distance[mask] = dist[mask]
        farthest = torch.max(distance, -1)[1]
    return centroids

def index_points(points, idx):
    """
    Input:
        points: input points data, [B, N, C]
        idx: sample index data, [B, S]
    Return:
        new_points:, indexed points data, [B, S, C]
    """
    device = points.device
    B = points.shape[0]
    view_shape = list(idx.shape)
    view_shape[1:] = [1] * (len(view_shape) - 1)
    repeat_shape = list(idx.shape)
    repeat_shape[0] = 1
    batch_indices = torch.arange(B, dtype=torch.long).to(device).view(view_shape).repeat(repeat_shape)
    new_points = points[batch_indices, idx, :]
    return new_points

def query_ball_point(radius, nsample, xyz, new_xyz):
    """
    Input:
        radius: local region radius
        nsample: max sample number in local region
        xyz: all points, [B, N, 3]
        new_xyz: query points, [B, S, 3]
    Return:
        group_idx: grouped points index, [B, S, nsample]
    """
    device = xyz.device
    B, N, C = xyz.shape
    _, S, _ = new_xyz.shape
    group_idx = torch.arange(N, dtype=torch.long).to(device).view(1, 1, N).repeat([B, S, 1])
    sqrdists = square_distance(new_xyz, xyz)
    group_idx[sqrdists > radius ** 2] = N
    group_idx = group_idx.sort(dim=-1)[0][:, :, :nsample]
    group_first = group_idx[:, :, 0].view(B, S, 1).repeat([1, 1, nsample])
    mask = group_idx == N
    group_idx[mask] = group_first[mask]
    return group_idx

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"


Set Abstraction

In [7]:
xyz, features = torch.rand(8, 1024, 3), torch.rand(8, 1024, 5)
print(xyz.shape, features.shape)

npoint, radius, nsample = 256, 0.1, 32

torch.Size([8, 1024, 3]) torch.Size([8, 1024, 5])


In [15]:
B, N, C = xyz.shape
S = npoint

fps_idx = farthest_point_sample(xyz, npoint)
print("furthese point sample idx:", fps_idx.shape)

new_xyz = index_points(xyz, fps_idx)
print("new xyz: ", new_xyz.shape)

idx = query_ball_point(radius, nsample, xyz, new_xyz)
print("query ball idx: ", idx.shape)

grouped_xyz = index_points(xyz, idx)
grouped_xyz = grouped_xyz - new_xyz.view(B, S, 1, C)
print("grouped xyz:", grouped_xyz.shape)

grouped_features = index_points(features, idx)
print("grouped features:", grouped_features.shape)

new_points = torch.cat([grouped_xyz, grouped_features], dim=-1)
print("new points: ", new_points.shape)

furthese point sample idx: torch.Size([8, 256])
new xyz:  torch.Size([8, 256, 3])
query ball idx:  torch.Size([8, 256, 32])
grouped xyz: torch.Size([8, 256, 32, 3])
grouped features: torch.Size([8, 256, 32, 5])
new points:  torch.Size([8, 256, 32, 8])


Feature Propagation

torch.Size([8, 128, 1])
torch.Size([8, 256, 1])


torch.Size([8, 3])